In [13]:
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import Model, Sequential

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

from tensorflow.keras.layers import Dense, Conv1D, LSTM, Lambda, Reshape, RNN, LSTMCell

import warnings
warnings.filterwarnings('ignore')

In [14]:
plt.rcParams['figure.figsize'] = (10, 7.5)
plt.rcParams['axes.grid'] = False

In [15]:
print(tf.__version__)

2.12.0


In [16]:
tf.random.set_seed(42)
np.random.seed(42)

In [17]:
df = pd.read_csv('data\BTC_final.csv')
df=df.drop(['Close','index'],axis=1)
df.head()

,Date,Open,High,Low,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD,ScorePositive,ScoreNegative,ScoreNeutral
0,2018-01-28,11475.299805,12040.299805,11475.299805,11786.299805,8.350360e+09,11259.571708,11635.535854,11516.047002,11884.539097,37.129744,0.560353,15192.188206,9606.291969,-761.452864,0.023502,0.057272,0.919226
1,2018-01-29,11755.500000,11875.599609,11179.200195,11296.400391,7.107360e+09,11311.714565,11455.293039,11461.122271,11804.864485,34.531168,0.835160,14866.144405,9602.435771,-744.090581,0.163158,0.294283,0.542558
2,2018-01-30,11306.799805,11307.200195,10036.200195,10106.299805,8.637860e+09,11202.843052,11356.421596,11122.356157,11575.251876,40.137091,1.000000,14450.867575,9531.012601,-816.944788,0.102038,0.260553,0.637410
3,2018-01-31,10108.200195,10381.599609,9777.419922,10221.099609,8.041160e+09,11040.228655,11287.314453,10897.011838,11392.534553,42.917380,1.000000,14318.165125,9345.245031,-855.556563,0.098884,0.268044,0.633072
4,2018-02-01,10237.299805,10288.799805,8812.280273,9170.540039,9.959400e+09,10741.820033,11122.717285,10465.350527,11093.196564,34.828472,0.651247,14132.400116,9050.004084,-959.863343,0.075772,0.452595,0.471633


In [7]:
df.shape

(1798, 18)

In [16]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Open,1798.0,2.044423e+04,1.710013e+04,3.236275e+03,7.632938e+03,1.081989e+04,3.424693e+04,6.754973e+04
High,1798.0,2.094373e+04,1.754835e+04,3.275378e+03,7.781352e+03,1.107785e+04,3.522661e+04,6.878962e+04
Low,1798.0,1.988138e+04,1.657819e+04,3.191303e+03,7.474553e+03,1.057076e+04,3.269781e+04,6.638206e+04
Adj Close,1798.0,2.044473e+04,1.709443e+04,3.236762e+03,7.627130e+03,1.081944e+04,3.419121e+04,6.756683e+04
Volume,1798.0,2.679515e+10,1.990957e+10,2.923670e+09,1.308705e+10,2.447217e+10,3.594142e+10,3.509679e+11
SMA_7,1798.0,2.043570e+04,1.706446e+04,3.351422e+03,7.633229e+03,1.074397e+04,3.423770e+04,6.551088e+04
SMA_14,1798.0,2.042550e+04,1.703161e+04,3.462801e+03,7.645625e+03,1.072002e+04,3.426376e+04,6.398306e+04
EMA_7,1798.0,2.043593e+04,1.704622e+04,3.350065e+03,7.609994e+03,1.072360e+04,3.433275e+04,6.473356e+04
EMA_14,1798.0,2.042699e+04,1.699269e+04,3.490708e+03,7.666117e+03,1.073217e+04,3.451715e+04,6.394564e+04
RSI,1798.0,5.108808e+01,1.823755e+01,4.699489e+00,3.725493e+01,5.088592e+01,6.412798e+01,9.553834e+01


In [8]:
#將時間轉換為數值，以方便後續可以運算
timestamp_s = pd.to_datetime(df['Date']).map(datetime.datetime.timestamp)
df['Coin_timestamp'] = timestamp_s

In [9]:
df = df.drop(['Date'], axis=1)
df.head()
# df.to_csv('TimestampBTC.csv')

,Open,High,Low,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD,ScorePositive,ScoreNegative,ScoreNeutral,Coin_timestamp
0,11475.299805,12040.299805,11475.299805,11786.299805,8.350360e+09,11259.571708,11635.535854,11516.047002,11884.539097,37.129744,0.560353,15192.188206,9606.291969,-761.452864,0.023502,0.057272,0.919226,1.517069e+09
1,11755.500000,11875.599609,11179.200195,11296.400391,7.107360e+09,11311.714565,11455.293039,11461.122271,11804.864485,34.531168,0.835160,14866.144405,9602.435771,-744.090581,0.163158,0.294283,0.542558,1.517155e+09
2,11306.799805,11307.200195,10036.200195,10106.299805,8.637860e+09,11202.843052,11356.421596,11122.356157,11575.251876,40.137091,1.000000,14450.867575,9531.012601,-816.944788,0.102038,0.260553,0.637410,1.517242e+09
3,10108.200195,10381.599609,9777.419922,10221.099609,8.041160e+09,11040.228655,11287.314453,10897.011838,11392.534553,42.917380,1.000000,14318.165125,9345.245031,-855.556563,0.098884,0.268044,0.633072,1.517328e+09
4,10237.299805,10288.799805,8812.280273,9170.540039,9.959400e+09,10741.820033,11122.717285,10465.350527,11093.196564,34.828472,0.651247,14132.400116,9050.004084,-959.863343,0.075772,0.452595,0.471633,1.517414e+09


In [20]:
#二值化收盤價，昨天比今天高就是跌，就為False
UpDown=[True]
for i in range(1,len(df['Adj Close'])):
    if df['Adj Close'][i]>df['Adj Close'][i-1]:
        UpDown.append(1)
    else:
        UpDown.append(-1)
df['UpDown'] = UpDown

In [21]:
df.to_csv('TimestampBTC.csv')

In [19]:
#將資料集分為70%訓練、20%驗證、10%測試
n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):-7]
test_df = df[-7:]

train_df.shape, val_df.shape, test_df.shape

((1258, 18), (533, 18), (7, 18))

In [14]:
#將值縮限在0~1之間，加速訓練時間
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_df)

train_df[train_df.columns] = scaler.transform(train_df[train_df.columns])
val_df[val_df.columns] = scaler.transform(val_df[val_df.columns])
test_df[test_df.columns] = scaler.transform(test_df[test_df.columns])

In [15]:
train_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Open,255.0,0.475870,0.324374,0.0,0.131029,0.561992,0.773590,1.0
High,255.0,0.482538,0.330615,0.0,0.130131,0.561767,0.790308,1.0
Low,255.0,0.477884,0.312671,0.0,0.140670,0.526522,0.757242,1.0
Close,255.0,0.472531,0.323701,0.0,0.130102,0.527627,0.772061,1.0
Volume,255.0,0.223899,0.140210,0.0,0.139343,0.209760,0.277317,1.0
ScorePositive,255.0,0.185835,0.139091,0.0,0.074279,0.177265,0.269369,1.0
ScoreNegative,255.0,0.460104,0.205251,0.0,0.342141,0.452662,0.583187,1.0
ScoreNeutral,255.0,0.395870,0.185510,0.0,0.296977,0.378257,0.473426,1.0
Coin_timestamp,255.0,0.500000,0.290379,0.0,0.250000,0.500000,0.750000,1.0


In [20]:
train_df.to_csv('data/train.csv')
val_df.to_csv('data/val.csv')
test_df.to_csv('data/test.csv')

In [21]:
train_df

,Open,High,Low,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD,ScorePositive,ScoreNegative,ScoreNeutral,Coin_timestamp
0,11475.299805,12040.299805,11475.299805,11786.299805,8.350360e+09,11259.571708,11635.535854,11516.047002,11884.539097,37.129744,0.560353,15192.188206,9606.291969,-761.452864,0.023502,0.057272,0.919226,1.517069e+09
1,11755.500000,11875.599609,11179.200195,11296.400391,7.107360e+09,11311.714565,11455.293039,11461.122271,11804.864485,34.531168,0.835160,14866.144405,9602.435771,-744.090581,0.163158,0.294283,0.542558,1.517155e+09
2,11306.799805,11307.200195,10036.200195,10106.299805,8.637860e+09,11202.843052,11356.421596,11122.356157,11575.251876,40.137091,1.000000,14450.867575,9531.012601,-816.944788,0.102038,0.260553,0.637410,1.517242e+09
3,10108.200195,10381.599609,9777.419922,10221.099609,8.041160e+09,11040.228655,11287.314453,10897.011838,11392.534553,42.917380,1.000000,14318.165125,9345.245031,-855.556563,0.098884,0.268044,0.633072,1.517328e+09
4,10237.299805,10288.799805,8812.280273,9170.540039,9.959400e+09,10741.820033,11122.717285,10465.350527,11093.196564,34.828472,0.651247,14132.400116,9050.004084,-959.863343,0.075772,0.452595,0.471633,1.517414e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,34665.566406,35937.566406,34396.476562,35287.781250,2.492431e+10,34681.205915,33843.568638,34562.634509,34647.003256,48.902596,1.000000,39331.098151,30440.722943,-925.194452,0.354141,0.613700,0.032159,1.625328e+09
1254,35284.343750,35284.343750,33213.660156,33746.003906,2.672155e+10,34582.872768,33991.376535,34358.476858,34526.870010,56.377266,1.000000,38179.280352,30926.514179,-922.970952,0.044931,0.561024,0.394045,1.625414e+09
1255,33723.507812,35038.535156,33599.917969,34235.195312,2.650126e+10,34349.646763,34114.914760,34327.656472,34487.980050,55.444150,0.957899,37503.766950,31190.840862,-871.686891,0.118939,0.491783,0.389278,1.625501e+09
1256,34225.679688,34997.664062,33839.289062,33855.328125,2.479603e+10,34180.288504,34124.364816,34209.574385,34403.626460,50.439630,0.732103,36771.198360,31503.591874,-851.876113,0.178382,0.213482,0.608136,1.625587e+09
